In [ ]:
import gilp
from ortools.linear_solver import pywraplp as OR

## Integer Program Examples

Consider the following 2D integer program:

$$\begin{align*}
\max \quad & 5x_1+8x_2\\
\text{s.t.} \quad & x_1 + x_2 \leq 6 \\
& 5x_1 + 9x_2 \leq 45 \\
& x_1, x_2 \geq 0, \quad \text{integer}
\end{align*}$$

In [ ]:
# Create the LP object
ip_2d = gilp.LP(A=[[1,1],
                   [5,9]],
                b=[6,45],
                c=[5,8])

In [ ]:
# Visualize branch and bound with manually selecting the branched on variable
gilp.bnb_visual(ip_2d, manual=True);

# Consider the following 5D integer program:

$$\begin{align*}
\mbox{minimize } x_1 + x_2 + x_3 + x_4 + x_5 + x_6 + x_7\\
\mbox{subject to }\qquad 
x_1 + x_4 + x_5 + x_6 + x_7 & \ge 17 \\
x_1 + x_2 + x_5 + x_6 + x_7 & \ge 13 \\
x_1 + x_2 + x_3 + x_6 + x_7 & \ge 15 \\
x_1 + x_2 + x_3 + x_4 + x_7 & \ge 19 \\
x_1 + x_2 + x_3 + x_4 + x_5 & \ge 14 \\
x_2 + x_3 + x_4 + x_5 + x_6 & \ge 16 \\
x_3 + x_4 + x_5 + x_6 + x_7 & \ge 11 \\
x_i & \ge 0, \ \ i=1,\ldots,7, \mbox{ integer}
\end{align*}$$

In [ ]:
def Example():
    # define model
    m = OR.Solver('ex', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # decision variables
    x = {}    
    for i in range(1,8):
        x[i] = m.NumVar(0, m.infinity(), 'x_%d' % (i)) 
        
    # define objective function here
    m.Minimize(sum(x[i] for i in range(1,8)))
    
    # constraints
    m.Add(x[1] + x[4] + x[5] + x[6] + x[7] >= 17)
    m.Add(x[1] + x[2] + x[5] + x[6] + x[7] >= 13)
    m.Add(x[1] + x[2] + x[3] + x[6] + x[7] >= 15)
    m.Add(x[1] + x[2] + x[3] + x[4] + x[7] >= 19)
    m.Add(x[1] + x[2] + x[3] + x[4] + x[5] >= 14)
    m.Add(x[2] + x[3] + x[4] + x[5] + x[6] >= 16)
    m.Add(x[3] + x[4] + x[5] + x[6] + x[7] >= 11)
    
    return (m, x)  # return the model and the decision variables

In [ ]:
def solve(m):
    m.Solve()
    
    print('Objective =', m.Objective().Value())
    print('iterations :', m.iterations())
    print('branch-and-bound nodes :',m.nodes())
    
    return ({var.name() : var.solution_value() for var in m.variables()})

In [ ]:
m, x = Example()
solve(m)

<img src="tree_1.png" width="500"/>

In [ ]:
m, x = Example()
m.Add(x[1] >= 2)
solve(m)

<img src="tree_2.png" width="700"/>

In [ ]:
m, x = Example()
m.Add(x[1] >= 2)
m.Add(x[2] >= 6)
solve(m)
# fathom by integrality
# this solution becomes the incumbent

<img src="tree_3.png" width="700"/>

In [ ]:
m, x = Example()
m.Add(x[1] >= 2)
m.Add(x[2] <= 5)
solve(m)
# fathom by bound

<img src="tree_4.png" width="700"/>

In [ ]:
m, x = Example()
m.Add(x[1] <= 1)
solve(m)
# fathom by bound

<img src="tree_5.png" width="700"/>